In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.version
#Import Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
import shutil
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics
import tensorflow.contrib.rnn as rnn
from random import shuffle

from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
#TF Version
tf.__version__

#with warnings.catch_warnings():
#    warnings.filterwarnings("ignore",category=FutureWarning)
#    import h5py

num_periods_output = 2 #to predict
num_periods_input=4 #input

No_Of_weeks=2

ALL_Test_Data=[]
ALL_Test_Prediction=[]


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## preprocessing

In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

def preprocessing(df_,num_features):
    
  # get year, month,Day,Hour,Minute
    df_['LastUpdated'] = pd.to_datetime(df_['LastUpdated'])
    df_['Year'] = df_['LastUpdated'].dt.year
    df_['Month'] = df_['LastUpdated'].dt.month
    df_['Day'] = df_['LastUpdated'].dt.day
    df_['Hour'] = df_['LastUpdated'].dt.hour
    df_['Minute'] = df_['LastUpdated'].dt.minute
    
    #,'Month','Day','Hour','Minute'
    # select features
    df=df_[['ID','Occ_percent','Month','Day','Hour','Minute',
           'temperature','dew_point','humidity','wind_speed',
           'feels_like','Events_Football_City','Events_Football_Derby',
           'Events_Football_Aston','Events_Rugby']]
    
    cols=df.columns
    print(cols[20:]) 
    
    ################################################encoding########################
    df['Occ_percent'] = pd.to_numeric(df['Occ_percent'],errors='coerce')
    df['Occ_percent'] = df['Occ_percent'].abs()
    
    Number_Of_Features=num_features
    df=df.values
    df = df.astype('float32')
    split=num_periods_output+num_periods_input
    
    ####################### CUT THE PORTION of the data that we are working on 
    
     ########################## SPLITTING FOR TESTING ##########################
    test_len=np.floor(len(df)*0.1)
    mod=test_len%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    test_len=int(test_len-mod)
    Test=df[(len(df)-test_len):,:]

    ########################### SPLITTING FOR TRAIN ###########################
    train_cut=int(np.floor(len(df)*0.2))
    new_cutted_df=df[:(len(df)-train_cut),:]
    Start_train_index=int(np.floor(2*24*7*No_Of_weeks))
    print('instances',Start_train_index)
    Train=new_cutted_df[len(new_cutted_df)-Start_train_index:,:]
    train_len=len(Train)
    mod=train_len%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    train_len=int(train_len-mod)
    Train=Train[0:train_len,:]
    print('len Train',len(Train))
   
        
    #############################  Normalization on train and validation separatly  #############
     
    ID_Train=Train[:,0]
    Train=np.delete(Train,[0],1)
    #x_batches=x_batches.drop(columns=['ID'], axis=1)
    occ_Train=Train[:,0]
    Train=np.delete(Train,[0],1)
    #x_batches=x_batches.drop(columns=['Occupancy'], axis=1)
    #normalizing data
    Train = Train.astype('float32')
    normalizer = MinMaxScaler().fit(Train)
    #normalizer = Normalizer().fit(Train)
    Train=normalizer.transform(Train)
    
    ID_Train=np.reshape(ID_Train,(len(ID_Train),1))
    occ_Train=np.reshape(occ_Train,(len(occ_Train),1))
    
    Train=np.append(occ_Train, Train, axis=1)
    Train=np.append(ID_Train, Train, axis=1)

    ############################################ TRAIN minibatches ##################################
    
    end=len(Train)
    start=0
    next=0
    x_batches=[]
    y_batches=[]
    
    count=0
    #print('lennnn',len(Train))
    while next+(num_periods_input+num_periods_output)<end:
        next=start+num_periods_input
        x_batches.append(Train[start:next,:])
        y_batches.append(Train[next:next+num_periods_output,1])
        start=start+1
    y_batches=np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)   
    x_batches=np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)   
    print('len x_batches ',len(x_batches))
    
    ###########################################TEST#####################################
    
    ID_Test=Test[:,0]
    Test=np.delete(Test,[0],1)
    #X_test=X_test.drop(columns=['ID'], axis=1)
    occ_Test=Test[:,0]
    Test=np.delete(Test,[0],1)
    #X_test=X_test.drop(columns=['Occupancy'], axis=1)
    Test = Test.astype('float32')
    Test=normalizer.transform(Test)
    
    #------------------
    ID_Test=np.reshape(ID_Test,(len(ID_Test),1))
    occ_Test=np.reshape(occ_Test,(len(occ_Test),1))
    
    Test=np.append(occ_Test,Test, axis=1)
    Test=np.append(ID_Test, Test, axis=1)
    
    ############################################ TEST minibatches ##################################
    end_test=len(Test)
    start_test=0
    next_test=0
    x_testbatches=[]
    y_testbatches=[]
    
    
    #print('lennnn',len(Train))
    while next_test+(num_periods_input+num_periods_output)<end_test:
        next_test=start_test+num_periods_input
        x_testbatches.append(Test[start_test:next_test,:])
        y_testbatches.append(Test[next_test:next_test+num_periods_output,1])
        start_test=start_test+1
    y_testbatches=np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)   
    x_testbatches=np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features) 
    print('len Test',len(Test))
    print('len xTestbatches',len(x_testbatches))
    ######################## Sampling##########################################
    
    #x_batches, y_batches, x_validbatches, y_validbatches, x_testbatches, y_testbatches
    
    return x_batches, y_batches, x_testbatches, y_testbatches

In [0]:
def load_locationfiles(path,loc_id):
    filename=path + '/Birm'+str(loc_id)+'.csv'
    print(filename)
    data_loc=pd.read_csv(filename)
    #mod=len(data_loc)%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    #data_loc=data_loc[:len(data_loc)-mod]
    return data_loc

In [0]:
    data_path=r'/content/drive/My Drive/Birm_Data_Events'
    #r'/home/shero/Desktop/OurProject/BanesData/Model/occupation_loc/'
    data_All=pd.DataFrame()
    x_batches_Full=[]
    y_batches_Full=[]
    X_Test_Full=[]
    Y_Test_Full=[]

    range_list = [x for x in range(1,31) if x != 8 and x != 21]

    for loc_id in range_list:
        #========
        data=load_locationfiles(data_path,loc_id)
        header=list(data.columns.values)
        data=pd.DataFrame(data,columns=header)
        x_batches, y_batches,X_Test,Y_Test=preprocessing(data,15)
        #===============================
        for element1 in (x_batches):
            x_batches_Full.append(element1)
            
        for element2 in (y_batches):
            y_batches_Full.append(element2)
                        
        for element5 in (X_Test):
            X_Test_Full.append(element5)
            
        for element6 in (Y_Test):
            Y_Test_Full.append(element6)
    #---------------------shuffle minibatches X and Y together-------------------------------------
    print(len(x_batches_Full),'     length of all file : ',len(y_batches_Full))
    combined = list(zip(x_batches_Full, y_batches_Full))
    random.shuffle(combined)
    shuffled_batch_features, shuffled_batch_y = zip(*combined)  

/content/drive/My Drive/Birm_Data_Events/Birm1.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm2.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm3.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm4.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm5.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 114
len xTestbatches 105
/content/drive/My Drive/Birm_Data_Events/Birm6.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 114
len xTestbatches 105
/content/drive/My Drive/Birm_Data_Events/Birm7.csv
I

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm10.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm11.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm12.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm13.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm14.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 102
len xTestbatches 93
/content/drive/My Drive/Birm_Data_Events/Birm15.csv
Index([], dtype='object')
instances 672
len Tra

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 114
len xTestbatches 105
/content/drive/My Drive/Birm_Data_Events/Birm18.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm19.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 114
len xTestbatches 105
/content/drive/My Drive/Birm_Data_Events/Birm20.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 120
len xTestbatches 111
/content/drive/My Drive/Birm_Data_Events/Birm22.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 120
len xTestbatches 111
/content/drive/My Drive/Birm_Data_Events/Birm23.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm24.csv
Index([], dtype='object')
instances 672
len Tr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm27.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm28.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm29.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
/content/drive/My Drive/Birm_Data_Events/Birm30.csv
Index([], dtype='object')
instances 672
len Train 672
len x_batches  663
len Test 126
len xTestbatches 117
18564      length of all file :  18564


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [0]:
import itertools
import numpy as np
#xgboost part
print(len(x_batches_Full))
All_Training_Instances=[]

IDs=np.arange(30)
Static_Features=np.zeros((30,30))
Static_Features[np.arange(30),IDs]=1
 
#=============== change each window into Instance =================================
for i in range(0,len(shuffled_batch_features)):
    hold=[]
    temp=[]
    for j in range(0,len(shuffled_batch_features[i])):
      #print(len(hold))
      
      if j==(len(shuffled_batch_features[i])-1):
          index=int(shuffled_batch_features[i][j][0])
          hold=np.concatenate((hold, Static_Features[index-1]), axis=None)
          temp=np.delete(shuffled_batch_features[i][j], [0], axis=0) 
          hold=np.concatenate((hold, temp), axis=None)
          
      else:
          hold=np.concatenate((hold, shuffled_batch_features[i][j][1]), axis=None)
          
    #print(len(hold))
    All_Training_Instances.append(hold)
    

print(len(All_Training_Instances[0]))

18564
47


In [0]:
#=================Testing=====================
All_Testing_Instances=[]

#=============== change each window into Instance =================================
print(len(X_Test_Full))
for i in range(0,len(X_Test_Full)):
  hold=[]
  temp=[]
  for j in range(0,len(X_Test_Full[i])):
       #print(len(hold))
      if j==(len(X_Test_Full[i])-1):
          index=int(X_Test_Full[i][j][0])
          hold=np.concatenate((hold, Static_Features[index-1]), axis=None)
          temp=np.delete(X_Test_Full[i][j], [0], axis=0) 
          hold=np.concatenate((hold, temp), axis=None)
      else:
          hold=np.concatenate((hold, X_Test_Full[i][j][1]), axis=None)
   
  All_Testing_Instances.append(hold)

#prediction=multioutput.predict(All_Testing_Instances)
print(len(All_Testing_Instances[0]))
#===========================calling MultiOutput XGoost=========================
All_Testing_Instances=np.reshape(All_Testing_Instances, (len(All_Testing_Instances),len(All_Testing_Instances[0])))
Y_Test_Full=np.reshape(Y_Test_Full, (len(Y_Test_Full),num_periods_output))

#========== reshape train ==============================
All_Training_Instances=np.reshape(All_Training_Instances, (len(All_Training_Instances),len(All_Training_Instances[0])))
shuffled_batch_y=np.reshape(shuffled_batch_y, (len(shuffled_batch_y),num_periods_output))



print(All_Training_Instances.shape)
model=xgb.XGBRegressor(learning_rate =0.1,
 n_estimators=800,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 nthread=4,
 scale_pos_weight=1,
 seed=27,silent=False)


multioutput=MultiOutputRegressor(model).fit(All_Training_Instances,shuffled_batch_y)


print('Fitting Done!')

prediction=multioutput.predict(All_Testing_Instances)
#print('prediction ',prediction.shape)
#print('test ',Y_Test_Full.shape)

MSE=np.mean(( prediction- Y_Test_Full)**2)

3180
47
(18564, 47)
[18:17:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


[18:17:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting Done!


In [0]:
print('With Features for {} weeks'.format(No_Of_weeks))
print('RMSE: ',MSE**0.5)

With Features for 2 weeks
RMSE:  4.491593455966561
